In [161]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib as plt

In [162]:
path = "./Resources/sample_base.csv"
fraud_df = pd.read_csv(path,header='infer',index_col=None)
fraud_df

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.1,0.771248,-1,192,20,0.008449,-1.050951,AB,2570,...,0,1500.0,0,INTERNET,37.831889,windows,0,1,0,2
1,0,0.9,0.868609,-1,8,30,0.003503,-0.683337,AB,1081,...,1,200.0,0,INTERNET,1.610226,linux,1,1,0,6
2,0,0.6,0.461974,30,8,30,0.006326,-0.916832,AB,2162,...,0,500.0,0,INTERNET,5.502750,linux,0,1,0,3
3,0,0.9,0.388111,31,2,50,0.010480,-0.817218,AB,2676,...,1,1500.0,0,INTERNET,3.209488,windows,1,1,0,4
4,0,0.9,0.512189,-1,249,50,0.013239,-0.230889,AB,621,...,1,500.0,0,INTERNET,8.416212,other,1,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,0.1,0.315213,-1,381,20,0.029162,-0.961046,AC,1774,...,0,200.0,0,INTERNET,8.994981,other,1,1,0,4
199996,0,0.7,0.546810,-1,79,40,0.010970,-1.193305,AC,1449,...,1,1500.0,0,INTERNET,2.285306,windows,1,0,0,0
199997,0,0.1,0.999241,28,13,20,0.013964,-1.608212,AD,1097,...,0,200.0,0,INTERNET,5.855588,other,1,1,0,0
199998,0,0.2,0.022199,-1,55,60,0.014597,-1.380137,AB,1766,...,0,200.0,0,INTERNET,9.292629,windows,0,2,0,1


In [163]:
fraud_df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.1,0.771248,-1,192,20,0.008449,-1.050951,AB,2570,...,0,1500.0,0,INTERNET,37.831889,windows,0,1,0,2
1,0,0.9,0.868609,-1,8,30,0.003503,-0.683337,AB,1081,...,1,200.0,0,INTERNET,1.610226,linux,1,1,0,6
2,0,0.6,0.461974,30,8,30,0.006326,-0.916832,AB,2162,...,0,500.0,0,INTERNET,5.502750,linux,0,1,0,3
3,0,0.9,0.388111,31,2,50,0.010480,-0.817218,AB,2676,...,1,1500.0,0,INTERNET,3.209488,windows,1,1,0,4
4,0,0.9,0.512189,-1,249,50,0.013239,-0.230889,AB,621,...,1,500.0,0,INTERNET,8.416212,other,1,1,0,7


In [164]:
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 32 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   fraud_bool                        200000 non-null  int64  
 1   income                            200000 non-null  float64
 2   name_email_similarity             200000 non-null  float64
 3   prev_address_months_count         200000 non-null  int64  
 4   current_address_months_count      200000 non-null  int64  
 5   customer_age                      200000 non-null  int64  
 6   days_since_request                200000 non-null  float64
 7   intended_balcon_amount            200000 non-null  float64
 8   payment_type                      200000 non-null  object 
 9   zip_count_4w                      200000 non-null  int64  
 10  velocity_6h                       200000 non-null  float64
 11  velocity_24h                      200000 non-null  f

In [165]:
fraud_df.columns

Index(['fraud_bool', 'income', 'name_email_similarity',
       'prev_address_months_count', 'current_address_months_count',
       'customer_age', 'days_since_request', 'intended_balcon_amount',
       'payment_type', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'employment_status',
       'credit_risk_score', 'email_is_free', 'housing_status',
       'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
       'has_other_cards', 'proposed_credit_limit', 'foreign_request', 'source',
       'session_length_in_minutes', 'device_os', 'keep_alive_session',
       'device_distinct_emails_8w', 'device_fraud_count', 'month'],
      dtype='object')

In [166]:
fraud_df['fraud_bool'].value_counts()

fraud_bool
0    197768
1      2232
Name: count, dtype: int64

In [167]:
object_df = fraud_df.select_dtypes(include=['object','string'])
object_df

,payment_type,employment_status,housing_status,source,device_os
0,AB,CA,BE,INTERNET,windows
1,AB,CA,BC,INTERNET,linux
2,AB,CA,BC,INTERNET,linux
3,AB,CB,BA,INTERNET,windows
4,AB,CA,BA,INTERNET,other
...,...,...,...,...,...
199995,AC,CA,BE,INTERNET,other
199996,AC,CA,BA,INTERNET,windows
199997,AD,CA,BE,INTERNET,other
199998,AB,CC,BD,INTERNET,windows


In [168]:
encode_payments = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

encode_payments.fit(object_df['payment_type'].values.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [169]:
encode_employment = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

encode_employment.fit(object_df['employment_status'].values.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [170]:
encode_housing = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

encode_housing.fit(object_df['housing_status'].values.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [171]:
encode_source = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

encode_source.fit(object_df['source'].values.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [172]:
encode_device = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

encode_device.fit(object_df['device_os'].values.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [173]:
numerical_df = fraud_df.select_dtypes(exclude=['object','string'])
numerical_df

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,...,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.1,0.771248,-1,192,20,0.008449,-1.050951,2570,6079.874351,...,1,11,0,1500.0,0,37.831889,0,1,0,2
1,0,0.9,0.868609,-1,8,30,0.003503,-0.683337,1081,1727.733739,...,1,1,1,200.0,0,1.610226,1,1,0,6
2,0,0.6,0.461974,30,8,30,0.006326,-0.916832,2162,1108.075600,...,0,1,0,500.0,0,5.502750,0,1,0,3
3,0,0.9,0.388111,31,2,50,0.010480,-0.817218,2676,3849.725772,...,1,19,1,1500.0,0,3.209488,1,1,0,4
4,0,0.9,0.512189,-1,249,50,0.013239,-0.230889,621,3282.921660,...,0,26,1,500.0,0,8.416212,1,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,0.1,0.315213,-1,381,20,0.029162,-0.961046,1774,10114.552068,...,1,-1,0,200.0,0,8.994981,1,1,0,4
199996,0,0.7,0.546810,-1,79,40,0.010970,-1.193305,1449,10863.678026,...,0,-1,1,1500.0,0,2.285306,1,0,0,0
199997,0,0.1,0.999241,28,13,20,0.013964,-1.608212,1097,7038.375489,...,1,26,0,200.0,0,5.855588,1,1,0,0
199998,0,0.2,0.022199,-1,55,60,0.014597,-1.380137,1766,8041.734623,...,1,5,0,200.0,0,9.292629,0,2,0,1


In [174]:
encoded_payments_df = pd.DataFrame(encode_payments.transform(object_df['payment_type'].values.reshape(-1, 1)), columns=encode_payments.get_feature_names_out())
encoded_employment_df = pd.DataFrame(encode_employment.transform(object_df['employment_status'].values.reshape(-1, 1)), columns=encode_employment.get_feature_names_out())
encoded_housing_df = pd.DataFrame(encode_housing.transform(object_df['housing_status'].values.reshape(-1, 1)), columns=encode_housing.get_feature_names_out())
encoded_source_df = pd.DataFrame(encode_source.transform(object_df['source'].values.reshape(-1, 1)), columns=encode_source.get_feature_names_out())
encoded_device_df = pd.DataFrame(encode_device.transform(object_df['device_os'].values.reshape(-1, 1)), columns=encode_device.get_feature_names_out())

dfs = [numerical_df, encoded_payments_df,encoded_employment_df,encoded_housing_df,encoded_source_df,encoded_device_df]

encoded_fraud_df = pd.concat(dfs,axis=1)

In [175]:
encoded_fraud_df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,...,x0_BE,x0_BF,x0_BG,x0_INTERNET,x0_TELEAPP,x0_linux,x0_macintosh,x0_other,x0_windows,x0_x11
0,0,0.1,0.771248,-1,192,20,0.008449,-1.050951,2570,6079.874351,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0.9,0.868609,-1,8,30,0.003503,-0.683337,1081,1727.733739,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0,0.6,0.461974,30,8,30,0.006326,-0.916832,2162,1108.075600,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0,0.9,0.388111,31,2,50,0.010480,-0.817218,2676,3849.725772,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0.9,0.512189,-1,249,50,0.013239,-0.230889,621,3282.921660,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [176]:
X = encoded_fraud_df.drop(columns="fraud_bool")
y =  encoded_fraud_df["fraud_bool"]

In [177]:
numerical_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())  # Normalize numerical features
])

In [178]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, X.columns)
    ])

In [179]:
# Apply preprocessing to the data
X_cleaned = preprocessor.fit_transform(X)

In [180]:
df_cleaned = pd.DataFrame(X_cleaned, columns=X.columns)
df_cleaned['fraud_bool'] = y.values

In [185]:

pca = PCA(random_state=42)
X_pca = pca.fit_transform(X_cleaned)
pca = PCA(n_components=0.90)
X_reduced = pca.fit_transform(X_cleaned)

df_reduced = pd.DataFrame(X_reduced)
df_reduced["fraud_bool"] = y.values
df_reduced.to_csv("Resources/PCA.csv",index=None)

In [186]:
df_reduced

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,fraud_bool
0,0.044850,0.867448,0.265031,1.061007,1.931991,2.603333,0.357424,-1.254235,-1.688642,-0.957369,...,0.963511,-0.816333,0.052802,-0.750628,1.078436,-1.183248,-1.714562,0.366764,0.522962,0
1,2.706031,-1.599373,0.344212,0.675761,-0.763198,-1.087848,1.460787,-0.151737,0.239009,1.254337,...,0.181613,0.633523,-0.706224,0.585980,-0.479103,0.460383,0.297016,1.030199,0.703719,0
2,0.230808,-0.546442,-0.568492,0.770089,-0.102704,-0.761389,1.895435,-0.434829,0.373212,-0.084965,...,0.038382,0.451016,1.477107,1.032601,1.768275,-1.267936,0.681954,-2.163354,0.626666,0
3,1.336715,2.274095,-0.687472,0.805492,1.150722,-0.335813,1.206365,3.425307,-0.470821,0.623067,...,0.270890,-0.506319,-1.123396,0.681636,0.924887,0.310248,0.479821,1.115171,-0.406148,0
4,3.923968,1.584103,0.663011,-0.066138,-0.166105,1.315669,0.052427,0.147390,1.427424,-0.100980,...,-0.362593,-0.370578,-0.509622,0.228207,-0.990007,-0.588697,-0.315169,0.371778,-0.359835,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.867461,-0.932826,0.239383,-0.878007,0.801785,3.246743,-2.897401,-1.026451,-0.101146,0.102903,...,-0.243540,-0.339057,0.777431,-1.459169,0.585925,0.199643,-1.203594,0.741851,-0.127226,0
199996,0.917905,3.084591,-1.131610,-0.899251,4.726554,-0.646631,-1.005487,-0.121406,0.683379,-0.063683,...,-0.162117,0.623444,-0.608187,-1.763318,0.023023,-4.097042,-0.898852,-0.129263,-0.044405,0
199997,-3.492213,-1.281898,0.869641,0.053085,0.635893,3.012325,0.030276,0.262639,0.147827,-0.409156,...,-0.263656,-0.286374,-0.227557,-1.198289,-0.643189,0.348691,0.493547,0.090156,-0.781423,0
199998,0.006483,1.985669,-3.447079,1.658523,-1.254661,-0.943087,-0.045389,-0.934788,-1.048134,-2.633759,...,-1.491129,-1.891168,-0.574815,-0.590384,0.839070,4.328488,1.006422,-0.786276,-0.025367,0
